In [2]:
%load_ext autoreload
%autoreload 2

In [18]:
import cv2
import wandb
import numpy as np
import time
import torch
from MTL.model import resnet,MTL_model
import glob
from torchvision.models import  resnet18
import torch.nn as nn

## single task (emotion,gender)

In [21]:
gender_model=resnet18()
gender_model.fc=nn.Linear(512,2)
gender_wt=torch.load('weight/classification/UTK_gender_best_model.pt')
emotion_model=resnet.EmotionModel(phase='test')
emotion_wt=torch.load('weight/classification/Exp128_resnet18_relu.pt')
device='cuda:3' if torch.cuda.is_available() else 'cpu'
#device='cpu'
gender_model.to(device)
emotion_model.to(device)

wandb.init(project='emotion+gender',entity='kookmin_ai')
wandb.run.name=(f'single_task_gpu_test')
example_images=[]
emo_label={0:'sad', 1:'happy', 2:'angry', 3:'disgust', 4:'surprise', 5:'fear', 6:'neutral'}
gender_label={0:'man',1:'woman'}
age_label={0:'youth', 1: 'student', 2: 'adult', 3:'elder'}

emotion_model.load_state_dict(emotion_wt)
gender_model.load_state_dict(gender_wt)
emotion_model.eval()
gender_model.eval()
test_imgs=glob.glob('testimgs/*.png')
for image in test_imgs:
    with torch.no_grad():

        img=cv2.imread(image)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        img=cv2.resize(img,(128,128))
        img=np.transpose(img,(2,0,1))
        img=torch.FloatTensor(img).to(device)
        img=torch.unsqueeze(img,0)/255.0
        
        start=time.time()
        emo_output=emotion_model(img)
        gender_output=gender_model(img)
        infer_time=time.time()-start
        
        emo_pred=emo_output.argmax(1,keepdim=True)
        gender_pred=gender_output.argmax(1,keepdim=True)
        emotion=emo_label[emo_pred.item()]
        gender=gender_label[gender_pred.item()]
    example_images.append(wandb.Image(
                    img, caption=f'Pred:{emotion} {gender}, inference_time:{infer_time:.4f}'))
wandb.log({"Image": example_images})

## multi task (emotion + gender)

In [22]:
model=MTL_model.MultiTaskModel(phase='test')
wt=torch.load('weight/MTL/resnet18_only_FC_best.pt')
device='cuda:3' if torch.cuda.is_available() else 'cpu'
# device='cpu'
model.to(device)

wandb.init(project='emotion+gender',entity='kookmin_ai')
wandb.run.name=(f'multi_task_gpu_test')
example_images=[]
emo_label={0:'sad', 1:'happy', 2:'angry', 3:'disgust', 4:'surprise', 5:'fear', 6:'neutral'}
gender_label={0:'man',1:'woman'}
age_label={0:'youth', 1: 'student', 2: 'adult', 3:'elder'}

model.load_state_dict(wt)
model.eval()
test_imgs=glob.glob('testimgs/*.png')
for image in test_imgs:
    with torch.no_grad():

        img=cv2.imread(image)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        img=cv2.resize(img,(128,128))
        img=np.transpose(img,(2,0,1))
        img=torch.FloatTensor(img).to(device)
        img=torch.unsqueeze(img,0)/255.0
        
        start=time.time()
        gender_output,emo_output=model(img)
        infer_time=time.time()-start
        
        emo_pred=emo_output.argmax(1,keepdim=True)
        gender_pred=gender_output.argmax(1,keepdim=True)
        emotion=emo_label[emo_pred.item()]
        gender=gender_label[gender_pred.item()]
    example_images.append(wandb.Image(
                    img, caption=f'Pred:{emotion} {gender}, inference_time:{infer_time:.4f}'))
wandb.log({"Image": example_images})

not pretrained model loaded
